# Preprocessing - Loading multiple images

In [9]:
import tensorflow as tf
import os
import numpy as np
from PIL import Image

- docs: https://www.tensorflow.org/api_docs/python/tf/WholeFileReader

In [10]:
# 현재 디렉토리를 이용해서 파일 이름을 확인한다.
# for dir in os.listdir(os.getcwd()):
#     print dir

In [11]:
image_path1 = os.getcwd() + '/image.png'
image_path2 = os.getcwd() + '/image-grayscaled.png'
filename_list = [image_path1, image_path2]
filename_queue = tf.train.string_input_producer(filename_list)

In [12]:
reader = tf.WholeFileReader()

key, value = reader.read(filename_queue)

print key
print value

Tensor("ReaderReadV2_1:0", shape=(), dtype=string)
Tensor("ReaderReadV2_1:1", shape=(), dtype=string)


- Encoding and Decoding : https://www.tensorflow.org/api_guides/python/image

In [13]:
images = tf.image.decode_png( value )

x = tf.cast(images, tf.float32)

# images_resized = tf.image.resize_images(images_decoded, [360, 360])


# Build Neural Network

- Reference : https://www.tensorflow.org/get_started/mnist/pros

## Placeholder

In [14]:
image_width = 721
image_height = 720

In [15]:
#x = tf.placeholder( tf.float32, shape=[None, image_width, image_width, 3]  )
y_ = tf.placeholder( tf.float32, shape=[None, 2] )


## Variables

In [16]:
# Convolution Layer 1
weight_hidden1 = tf.Variable( tf.truncated_normal([5, 5, 3, 32]) )
bias_hidden1 = tf.Variable( tf.zeros([32]) )

x_image = tf.reshape(x, [-1, image_width, image_height,3])

conv1 = tf.nn.conv2d(x_image, weight_hidden1, strides=[1,1,1,1], padding='SAME')

# ReLu
hidden1_layer = tf.nn.relu(conv1 + bias_hidden1)


In [17]:
# Convolution Layer 2
weight_hidden2 = tf.Variable(tf.truncated_normal([5, 5, 3, 64]))
bias_hidden2 = tf.Variable(tf.zeros([64]))

conv2 = tf.nn.conv2d(x_image, weight_hidden2, strides=[1, 1, 1, 1], padding='SAME')

# ReLu
hidden2_layer = tf.nn.relu(conv2 + bias_hidden2)


In [18]:
# Densely Connected Layer 
weight_fully_connected = tf.Variable(tf.truncated_normal([image_width*image_height*64, 1024]))
bias_fully_connected = tf.Variable(tf.zeros([1024]))

hidden_pool2_flat = tf.reshape(hidden2_layer, [-1, image_width*image_height*64])
hidden_fully_connected = tf.nn.relu(tf.matmul(hidden_pool2_flat, weight_fully_connected)+bias_fully_connected)


# Preprocessing - Test for showing the images

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt

with tf.Session() as sess:
    coord = tf.train.Coordinator()
    thread = tf.train.start_queue_runners(sess, coord)

    # 입력받은 이미지를 출력한다.
    # fig = plt.figure()

    # for i in range(len(filename_list)):
    #     image = sess.run( images )

    #     original = Image.fromarray(image)
    #     fig.add_subplot(2,1,i+1)
    #     imgplot = plt.imshow(np.asarray(original))

    #     print i

    # CNN학습을 진행한다.
    sess.run(tf.global_variables_initializer())
    sess.run(hidden_fully_connected)

    coord.request_stop()
    coord.join()

    print 'Convolution Network Training is done'

sess.close()

- 컴퓨터의 벡터 연산과 수학에서 표현하는 벡터간의 표현차이

In [0]:
import numpy as np

vector_in_computer = [[1,2,3,4,5], [6,7,8,9,10]]
print vector_in_computer
print np.shape(vector_in_computer)

vector_in_math = [[1], [2], [3], [4], [5]]
print vector_in_math
print np.shape(vector_in_math)
